In [1]:
import pandas as pd
import numpy as np
import os
import json
import csv
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Bidirectional, Embedding, LSTM
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras import backend as kb
from keras import initializers
from keras.callbacks import ReduceLROnPlateau
from keras.models import load_model
#!pip install python_highcharts
from highcharts import Highchart
import json

Using TensorFlow backend.


In [2]:
def getJsonList(filePath):
    fileList = []
    for i,j,k in os.walk(filePath):
        fileList = k
    return fileList
def calcWeight(item):
    zeros = 0
    for i in item['pose_keypoints_2d']:
        if i == 0:
            zeros += 1
    for i in item['face_keypoints_2d']:
        if i == 0:
            zeros +=1
    return zeros

def getJsonForOneFrame(fileName, filePath):
    l = getJsonList(filePath)
    with open(fileName,'r') as f:
        temp = json.loads(f.read())
        allPeople = temp['people']
    if len(allPeople) == 0:
        empty = []
        for i in range(190):
            empty.append(0)
        return empty
    target = allPeople[0]
    for dic in allPeople:
        if calcWeight(dic) < calcWeight(target):
            target = dic
    matrix = []
    if(len(target['pose_keypoints_2d']) == 0):
        for i in range(50):
            matrix.append(0)
    else:
        for i in range(75):
            if (i+1) % 3 == 0:
                pass
            else:
                matrix.append(target['pose_keypoints_2d'][i])
    if(len(target['face_keypoints_2d']) == 0):
        for i in range(140):
            matrix.append(0)
    else:
        for i in range(210):
            if (i+1) % 3 == 0:
                pass
            else:
                matrix.append(target['face_keypoints_2d'][i])
    return matrix


def writeData(path, pathType,fullDfList):
    fileList = getJsonList(path)
    fileList = set(fileList)
    print(len(fileList))
    dataCat = pd.read_csv('data.csv')
    for index, row in dataCat.iterrows():
        temp = int(row['name'][:-1])
        if(row["type"] != pathType):
            pass
        else:
            startName = row["name"]+"_"+'0'.zfill(12)+"_keypoints.json"
            endName = row["name"]+"_"+'30'.zfill(12)+"_keypoints.json"
            if endName and endName not in fileList:
                print('Video ',row["name"]," is not long enough")
            else:
                initFrame = 0
                while(row["name"]+"_"+str(initFrame + 29).zfill(12)+"_keypoints.json" in fileList):
                    matrix = []
                    for i in range(30):
                        if (i+1)% 5 == 0:
                            file = row["name"]+"_"+str(initFrame + i).zfill(12)+"_keypoints.json"
                            matrixLine = getJsonForOneFrame(path+'/'+file,path)
                            matrix.append(matrixLine)
                        else:
                            pass
                    if row['type'] == 'shortyes':
                        label = True
                    elif row['type'] == 'shortno':
                        label = False
                    else:
                        label = getLable(dataCat,row['name'], initFrame)
                    data = [[label, matrix]]
                    print("Data:",label, row['name'], initFrame,'matrix:',len(matrix),len(matrix[0]))
                    df = pd.DataFrame(data,columns=['lable','matrix'])
                    fullDfList.append(df)
                    initFrame += 30
    return fullDfList

def draw(folderName,videoName, modelName):
    #videoName = '6'#define your test video name here
    fileList = getJsonList(folderName)
    fileList = set(fileList)
    print(len(fileList))
    fullData = []
    initFrame = 0
    while(folderName+"_"+str(initFrame + 29).zfill(12)+"_keypoints.json" in fileList):
        matrix = []
        for i in range(30):
            file = folderName+"_"+str(initFrame + i).zfill(12)+"_keypoints.json"
            matrixLine = getJsonForOneFrame(folderName+'/'+file,folderName)

            matrix.append(matrixLine)
        fullData.append(matrix)
        initFrame += 30
    def cleanX(raw):
        length = len(raw)
        temp = np.zeros((length, 10,190))
        for i in range(len(raw)):
            for j in range(10):
                for k in range(190):
                    temp[i][j][k] = raw[i][j][k]
            print("progress:{0}%".format(round((i + 1) * 100 / len(raw))), end="\r")
        return temp
    temp = cleanX(fullData)
    
    model1 = load_model('balancetalking.h5')
    r1 = model1.predict(temp)
    
    
    model2 = load_model('balancedrinking.h5')
    r2 = model2.predict(temp)
    
    model3 = load_model('balanceeating.h5')
    r3 = model3.predict(temp)
    
    X_TRAIN = []
    for i in range(len(r1)):
        X_TRAIN.append([[r1[i][0]], [r2[i][0]], [r3[i][0]]])
    X_TRAIN = np.array(X_TRAIN)
    print(X_TRAIN.shape)
    
    model = load_model(modelName)
    predict = model.predict(X_TRAIN)
    
    
    a = predict.tolist()
    result = []
    frames = 30
    for i in a:
        result.append([frames,i[0]])
        frames += 30
    #print(result)
    chart = Highchart()
    chart.set_options('chart', {'inverted': False})
    options = {
        'title': {
            'text': 'Prediction for video '+folderName+'.mp4'
        },
        'subtitle': {
            'text': '1 means talking while 0 means non-talking'
        },
         'xAxis': {
            'title': {
                'text': 'Second'
            }
        },
        'yAxis': {
            'title': {
                'text': 'Flag'
            },
        }
    }
    chart.set_dict_options(options)
    chart.add_data_set(result, series_type='line', name='prediction')
    
    
    
    index = []
    for i in result:
        for j in range(30):
            index.append(i[1])
            
        
    import cv2
    video = videoName
    result_video = folderName+'_result.mp4'
    cap = cv2.VideoCapture(video)
    fps_video = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    videoWriter = cv2.VideoWriter(result_video, fourcc, fps_video, (frame_width, frame_height))
    frame_id = 0
    while (cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            frame_id += 1
            left_x_up = int(frame_width / frame_id)
            left_y_up = int(frame_height / frame_id)
            right_x_down = int(left_x_up + frame_width / 10)
            right_y_down = int(left_y_up + frame_height / 10)
            word_x = left_x_up + 5
            word_y = left_y_up + 25
            if frame_id >= len(index):
                pass
            else:
                cv2.putText(frame, 'frame_%s' %index[frame_id], (word_x, word_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (55,255,155), 2)
            videoWriter.write(frame)
        else:
            videoWriter.release()
            break    
    
    frameRate = 30#default video frame rate
    for i in result:
        i[0] = i[0]/frameRate
    print(result)
    
    d = {}
    d['talking'] = result
    file_name = 'timeLabel_'+videoName+'.json' 
    with open(file_name,'w') as file_object:
        json.dump(d,file_object)
    return chart

In [4]:
folder_name = '5'#type your folder path here, which contains openpose json files
video_name = '5.mp4'#type your video path here, we want to know frame rate to create time labels 
model_name = 'ens84.h5'#type your model name here
chart1 = draw(folder_name,video_name,model_name)#return a frame/label chart and saved a time/label json file
chart1.save_file('chart_for_video_'+video_name)#save the chart 
chart1

1816
(60, 3, 1)00%
[[1.0, 0.007347702980041504], [2.0, 0.007351309061050415], [3.0, 0.007346838712692261], [4.0, 0.00721701979637146], [5.0, 0.00721016526222229], [6.0, 0.007318019866943359], [7.0, 0.007262855768203735], [8.0, 0.0072509050369262695], [9.0, 0.007253378629684448], [10.0, 0.007211089134216309], [11.0, 0.0072141289710998535], [12.0, 0.007269680500030518], [13.0, 0.007242649793624878], [14.0, 0.007220655679702759], [15.0, 0.0072457194328308105], [16.0, 0.007279545068740845], [17.0, 0.007303714752197266], [18.0, 0.007705479860305786], [19.0, 0.007269531488418579], [20.0, 0.0073150694370269775], [21.0, 0.007357418537139893], [22.0, 0.0074336230754852295], [23.0, 0.007345348596572876], [24.0, 0.0074748694896698], [25.0, 0.0073115527629852295], [26.0, 0.007172584533691406], [27.0, 0.007170647382736206], [28.0, 0.007157444953918457], [29.0, 0.007158875465393066], [30.0, 0.007211536169052124], [31.0, 0.007178157567977905], [32.0, 0.0072977542877197266], [33.0, 0.00724777579307556